Need to create the Mutual Pointwise Information (MPI) matrices.

In [1]:
# read in the batched pickle cooccurrence matrices

cooccurPath = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/halfcentury/'

import pickle
from os import listdir
from os.path import isfile, join

with open('./IDToWord.p', 'rb') as fp:
    IDToWord = pickle.load(fp)
    
with open('./WordToID.p', 'rb') as fp:
    wordToID = pickle.load(fp)

cooccurFiles = [f for f in listdir(cooccurPath) if isfile(join(cooccurPath, f))]

In [2]:
batches = []
for cFile in cooccurFiles:
    with open("./halfcentury/" + cFile, "rb") as fp:
        batches.append(pickle.load(fp))

In [3]:
import numpy as np
'''
currently, matrices are X[word][coword] = 1,
so X[word_a][word_b] != X[word_b][word_a], 
since if a word is not included in the window it won't show up;
e.g. if the string is "a b c d e f g" and L = 3,
(d, [a b c d e f g])
but
a won't get a pairing, since it doesn't have a left window; thus
X[d,a] != X[a,d]

this is really the approximate cooccurence, and it's really only 
inaccurate for the first and last 5 words in a document.
'''
meanBatches = [(b + b.T)/2.0 for b in batches]

In [53]:
from scipy.sparse import csr_matrix
# X = meanBatches[0]
f = 10000
X = csr_matrix(np.random.randint(0,f, (f,f)))

In [54]:
# o = np.ones(len(wordToID))
o = np.ones(f)
S = X.dot(o)

In [55]:
X.setdiag(S)

/Users/bkitano/anaconda3/lib/python3.7/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [56]:
trace = np.sum(X.diagonal())

In [57]:
import time
from scipy.special import xlogy
# now X is the thing we want, discretely. so we have to do the scaling.
# want log (X[i,j] * trace(X) / X[i]X[j])
# log(X[i,j]) - log(X[i]) - log(X[j]) + log(trace(X))
# ETA for xlogy: 9578 seconds = 2h40m

a = time.time()
logXij = xlogy(np.sign(X.todense()), X.todense())
print(time.time() - a)
print(logXij.shape)

3.875932216644287
(10000, 10000)


In [58]:
b = time.time()
Xj = np.tile(S, [f, 1])
logXj = xlogy(np.sign(Xj), Xj)
print(time.time() - b)

3.0980331897735596


In [59]:
c = time.time()
logXi = logXj.T
print(time.time() - c)

0.0017750263214111328


In [60]:
d = time.time()
trace = np.trace(X.todense())
logTraceX = np.ones((f, f))*np.log(trace)
print(time.time() - d)

1.2689318656921387


In [61]:
e = time.time()
PMI = logXij + logTraceX - logXi - logXj
print(time.time() - e)

5.707242965698242


In [65]:
PPMI = np.multiply(PMI, np.greater(PMI, np.zeros((f,f))))

In [70]:
import pickle
with open('./ppmi/slice{}.p'.format('test'), 'wb') as fp:
    pickle.dump(PPMI, fp)